# S&P 500 Earning Revisions Breadth
© 2025 Jim Domeij & Marek Ozana

This calculates, for each GICS sector within the S&P 500, the number of companies with upward or downward EPS revisions over the past month. It can be used to track the breadth of change in analysts’ EPS estimates by sector.

In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/macro") / "spx-earning-revisions-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [2]:
# Get counts of comps with positive and negative earnings revisions over the past month
from polars_bloomberg import BQuery

query = """
    let(
        #rev = net_chg(is_eps(fpt=BT, fpo=1, ae=e, dates=range(-1M, 0d)));
        #sector = classification_name(gics, 1);

        #up   = sum(group(if(#rev > 0, 1, 0), #sector));
        #down = sum(group(if(#rev < 0, 1, 0), #sector));
    )
    get(#up, #down)
    for(members("SPX Index"))
    preferences(dropCols=["ORIG_IDS", "#sector"])
    """

with BQuery() as bq:
    df = bq.bql(query).combine()
df.head()

ID,#up,#down
str,f64,f64
"""Communication Services""",18.0,3.0
"""Consumer Discretionary""",41.0,8.0
"""Consumer Staples""",29.0,7.0
"""Energy""",12.0,10.0
"""Financials""",65.0,9.0


In [3]:
# Chart the results
import altair as alt
import polars as pl

g_data = df.select(
    pl.col("ID"),
    pl.col("#up").alias("up"),
    (-pl.col("#down")).alias("down"),
    (pl.col("#up") - pl.col("#down")).alias("net"),
)

base = alt.Chart(g_data, title="S&P 500 Earnings Revisions Breadth").encode(
    y=alt.Y("ID:N").title(None)
)

bars = (
    base.transform_fold(["up", "down"])
    .mark_bar()
    .encode(
        x=alt.X("value:Q").title("# of revisions last month"),
        color=alt.Color("key:N")
        .title(None)
        .sort(["up", "down"])
        .legend(orient="bottom"),
    )
)

net_tick = base.mark_tick(color="black", thickness=2, size=18).encode(
    x=alt.X("net:Q"),
)

chart = (bars + net_tick).properties(width=600, height=400)

# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=2)
